# [실습3] IMDB 영화 후기 데이터셋 벡터화

IMDB 데이터셋을 직접 다운로드하여 벡터화하는 과정을 살펴본다.

준비 과정 1: 데이터셋 다운로드 압축 풀기

압축을 풀면 아래 구조의 디렉토리가 생성된다.

```
aclImdb/
...train/
......pos/
......neg/
...test/
......pos/
......neg/
```

`train`의 `pos`와 `neg` 서브디렉토리에 각각 12,500개의 긍정과 부정 후기가
포함되어 있다.

In [1]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  4301k      0  0:00:19  0:00:19 --:--:-- 4221k


`aclImdb/train/unsup` 서브디렉토리는 필요 없기에 삭제한다.

In [2]:
import platform

if platform.system() == 'Linux':
    !rm -r aclImdb/train/unsup
else:
    import shutil
    unsup_path = './aclImdb/train/unsup'
    shutil.rmtree(unsup_path)

긍정 후기 하나의 내용을 살펴보자.
모델 구성 이전에 훈련 데이터셋을 살펴 보고
모델에 대한 직관을 갖는 과정이 항상 필요하다.

In [3]:
if 'google.colab' in str(get_ipython()):
    !cat aclImdb/train/pos/4077_10.txt
else:
    with open('aclImdb/train/pos/4077_10.txt', 'r') as f:
        text = f.read()
        print(text)

I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drink and food in reach so you don't have to stop the film.<br /><br />Enjoy


준비 과정 2: 검증셋 준비

훈련셋의 20%를 검증셋으로 떼어낸다.
이를 위해 `aclImdb/val` 디렉토리를 생성한 후에
긍정과 부정 훈련셋 모두 무작위로 섞은 후 그중 20%를 검증셋 디렉토리로 옮긴다.

In [4]:
import os, pathlib, shutil, random

base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"

for category in ("neg", "pos"):
    os.makedirs(val_dir / category)            # val 디렉토리 생성
    files = os.listdir(train_dir / category)

    random.Random(1337).shuffle(files)         # 훈련셋 무작위 섞기

    num_val_samples = int(0.2 * len(files))    # 20% 지정 후 검증셋으로 옮기기
    val_files = files[-num_val_samples:]

    for fname in val_files:
        shutil.move(train_dir / category / fname,
                    val_dir / category / fname)

준비 과정 3: 텐서 데이터셋 준비

`text_dataset_from_directory()` 함수를 이용하여
훈련셋, 검증셋, 테스트셋을 준비한다.
자료형은 모두 `Dataset`이며, 배치 크기는 32를 사용한다.

In [5]:
from tensorflow import keras

batch_size = 32

train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size
    )

val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
    )

test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
    )

Found 20000 files belonging to 2 classes.


2025-06-02 10:31:28.682533: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2025-06-02 10:31:28.682564: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2025-06-02 10:31:28.682574: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
2025-06-02 10:31:28.682602: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-06-02 10:31:28.682619: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


각 데이터셋은 배치로 구분되며
입력은 `tf.string` 텐서이고, 타깃은 `int32` 텐서이다.
크기는 모두 32이며 지정된 배치 크기이다.
예를 들어, 첫째 배치의 입력과 타깃 데이터의 정보는 다음과 같다.

In [6]:
for inputs, targets in train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)

    # 예제: 첫째 배치의 첫째 후기
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])

    break

inputs.shape: (32,)
inputs.dtype: <dtype: 'string'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor(b'While Urban Cowboy did not ooze with the same testosterone you might find at a rodeo, it did provide an accurate glimpse of that day and age, in urban Texas. I also think that to truly critique this movie, one would have to have lived in the time and relative place that it was made. There was good music, fun times and, yes, a few "rough and tumbles" at the honky tonk roadhouses. The relationship of Bud and Sissy, like "two ships passing in the night", was well conceived. When Pam tore up the note that Sissy had written to Bud, it echoed the tragedy of many true life romances. The entire story was well thought out. I thought the cast and crew did an excellent job. I thought the screen play was well written and directed. Scott Glenn should have received an Oscar for best supporting actor.', shape=(), dtype=string)
targets[0]: tf.Tensor(1, shape=(), dtype=int32)


### 11.3.3 시퀀스 활용법

**정수 벡터 데이터셋 준비**

훈련셋의 모든 후기 문장을 정수들의 벡터로 변환한다.
단, 후기 문장이 최대 600개의 단어만 포함하도록 한다.
또한 사용되는 어휘는 빈도 기준 최대 2만 개로 제한한다.

- `max_length = 600`
- `max_tokens = 20000`
- `output_sequence_length=max_length`

In [7]:
from tensorflow.keras import layers

max_length = 600
max_tokens = 20000

text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)
# 어휘 색인 생성 대상 훈련셋 후기 텍스트 데이터셋
text_only_train_ds = train_ds.map(lambda x, y: x)

text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y))
int_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y))
int_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y))

2025-06-02 10:32:03.773242: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


변환된 첫째 배치의 입력과 타깃 데이터의 정보는 다음과 같다.
`output_sequence_length=600`으로 지정하였기에 모든 문장은 단어를 최대 600개에서
잘린다. 따라서 생성되는 정수들의 벡터는 길이가 모두 600으로 지정된다.
물론 문장이 600개보다 적은 수의 단어를 사용한다면 나머지는 0으로 채워진다.
또한 벡터에 사용된 정수는 2만보다 작은 값이며,
이는 빈도가 가장 높은 2만개의 단어만을 대상(`max_tokens=20000`)으로 했기 때문이다.

In [9]:
for inputs, targets in int_train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])
    break

inputs.shape: (32, 600)
inputs.dtype: <dtype: 'int64'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor(
[ 5290  1041    21   123   200   962  1125     5   346    51   148    11
   959   420    69   362   470    46     6   391    34  3507     3  4037
  1853   762    41    29     5     2    88  1201     3     1   832     1
  7982   122    92    70  1239   143  5519     6     1    16     2  1618
    80    10   151  1355     6    82    98  2197    31    32     3   317
  6847     4   929     3 13241   398   860  1227    17     4 16994     8
     4  1970 11735  2249 13702    81   361     2    19   157  6455    43
     4 10652   739  1280     1   813  2217    17   959   475     8     4
  2207  3000  3263     3  7741     9     6    28    25  4685     6  8081
    21   959  3901    37  1000  2285     3    37  1000  4686 11640  1292
    32   629    51    27  5168  1970   466    25  4848     8     4  2249
   513   466     1    25     1  1302  2898    31     4     1   180 

**트랜스포머 구현**

위 그림에서 설명된 트랜스포머 인코더를 층(layer)으로 구현하면 다음과 같다.

In [10]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

**트랜스포머 인코더 활용 모델**

훈련 데이터셋이 입력되면 먼저 단어 임베딩을 이용하여
단어들 사이의 연관성을 찾는다.
이후 트랜스포머 인코더로 셀프 어텐션을 적용한다.

In [13]:
vocab_size = 20000
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")

x = layers.Embedding(vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

# 길이가 600인 1차원 어레이로 변환
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)

outputs = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs, outputs)

model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])

model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_2 (Embedding)     (None, None, 256)         5120000   
                                                                 
 transformer_encoder_2 (Tra  (None, None, 256)         543776    
 nsformerEncoder)                                                
                                                                 
 global_max_pooling1d_2 (Gl  (None, 256)               0         
 obalMaxPooling1D)                                               
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 dense_8 (Dense)             (None, 1)                 257 

훈련 과정은 특별한 게 없다.
테스트셋에 대한 정확도가 87.5% 정도로 바이그램 모델보다 좀 더 낮다.

In [14]:
callbacks = [
    keras.callbacks.ModelCheckpoint("transformer_encoder",
                                    save_best_only=True)
]

model.fit(int_train_ds, validation_data=int_val_ds, epochs=20, callbacks=callbacks)

model = keras.models.load_model(
    "transformer_encoder",
    custom_objects={"TransformerEncoder": TransformerEncoder})

print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/20


2025-06-02 10:32:21.739187: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


625/625 [==============================] - ETA: 0s - loss: 0.4676 - accuracy: 0.7875

2025-06-02 10:36:26.860206: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


INFO:tensorflow:Assets written to: transformer_encoder/assets


...

625/625 [==============================] - 266s 425ms/step - loss: 0.3132 - accuracy: 0.8679 - val_loss: 0.2684 - val_accuracy: 0.8892
Epoch 3/20
625/625 [==============================] - 265s 424ms/step - loss: 0.2685 - accuracy: 0.8900 - val_loss: 0.2713 - val_accuracy: 0.8872
Epoch 4/20
625/625 [==============================] - 262s 420ms/step - loss: 0.2258 - accuracy: 0.9118 - val_loss: 0.3955 - val_accuracy: 0.8442
Epoch 5/20
625/625 [==============================] - ETA: 0s - loss: 0.1904 - accuracy: 0.9255INFO:tensorflow:Assets written to: transformer_encoder/assets


INFO:tensorflow:Assets written to: transformer_encoder/assets


625/625 [==============================] - 263s 422ms/step - loss: 0.1904 - accuracy: 0.9255 - val_loss: 0.2508 - val_accuracy: 0.9014
Epoch 6/20
625/625 [==============================] - 265s 424ms/step - loss: 0.1546 - accuracy: 0.9413 - val_loss: 0.3495 - val_accuracy: 0.8760
Epoch 7/20
625/625 [==============================] - 265s 424ms/step - loss: 0.1234 - accuracy: 0.9539 - val_loss: 0.3344 - val_accuracy: 0.8814
Epoch 8/20
625/625 [==============================] - 259s 415ms/step - loss: 0.1032 - accuracy: 0.9603 - val_loss: 0.3354 - val_accuracy: 0.8836
Epoch 9/20
625/625 [==============================] - 260s 416ms/step - loss: 0.0817 - accuracy: 0.9700 - val_loss: 0.3590 - val_accuracy: 0.8868
Epoch 10/20
625/625 [==============================] - 268s 429ms/step - loss: 0.0672 - accuracy: 0.9754 - val_loss: 0.3739 - val_accuracy: 0.8920
Epoch 11/20
625/625 [==============================] - 269s 430ms/step - loss: 0.0568 - accuracy: 0.9800 - val_loss: 0.3874 - val_accu

2025-06-02 12:00:46.849385: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


782/782 [==============================] - 126s 161ms/step - loss: 0.3170 - accuracy: 0.8636
Test acc: 0.864


**단어 위치 인코딩**

다음 `PositionalEmbedding` 층 클래스는 두 개의 임베딩 클래스를 사용한다.
하나는 보통의 단어 임베딩이며,
다른 하나는 단어의 위치 정보를 임베딩한다.
각 임베딩의 출력값을 합친 값을 트랜스포머에게 전달하는 역할을 수행한다.

In [15]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

**단어위치인식 트랜스포머 아키텍처**

아래 코드는 `PositionalEmbedding` 층을 활용하여 트랜스포머 인코더가
단어위치를 활용할 수 있도록 한다.

In [16]:
vocab_size = 20000
sequence_length = 600
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")

x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)

outputs = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs, outputs)

In [17]:
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])

model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, None)]            0         
                                                                 
 positional_embedding (Posi  (None, None, 256)         5273600   
 tionalEmbedding)                                                
                                                                 
 transformer_encoder_3 (Tra  (None, None, 256)         543776    
 nsformerEncoder)                                                
                                                                 
 global_max_pooling1d_3 (Gl  (None, 256)               0         
 obalMaxPooling1D)                                               
                                                                 
 dropout_3 (Dropout)         (None, 256)               0         
                                                           

In [18]:
callbacks = [
    keras.callbacks.ModelCheckpoint("full_transformer_encoder",
                                    save_best_only=True)
]

model.fit(int_train_ds, validation_data=int_val_ds, epochs=20, callbacks=callbacks)

model = keras.models.load_model(
    "full_transformer_encoder",
    custom_objects={"TransformerEncoder": TransformerEncoder,
                    "PositionalEmbedding": PositionalEmbedding})
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/20


2025-06-02 12:10:27.631887: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


625/625 [==============================] - ETA: 0s - loss: 0.4762 - accuracy: 0.7775

2025-06-02 12:14:44.913449: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


INFO:tensorflow:Assets written to: full_transformer_encoder/assets


INFO:tensorflow:Assets written to: full_transformer_encoder/assets


625/625 [==============================] - 285s 452ms/step - loss: 0.4762 - accuracy: 0.7775 - val_loss: 0.2552 - val_accuracy: 0.8896
Epoch 2/20
625/625 [==============================] - 283s 453ms/step - loss: 0.2469 - accuracy: 0.9023 - val_loss: 0.2701 - val_accuracy: 0.9020
Epoch 3/20
625/625 [==============================] - 276s 441ms/step - loss: 0.1798 - accuracy: 0.9306 - val_loss: 0.3081 - val_accuracy: 0.9004
Epoch 4/20
625/625 [==============================] - 276s 442ms/step - loss: 0.1461 - accuracy: 0.9429 - val_loss: 0.4002 - val_accuracy: 0.8898
Epoch 5/20
625/625 [==============================] - 278s 445ms/step - loss: 0.1213 - accuracy: 0.9553 - val_loss: 0.3918 - val_accuracy: 0.8932
Epoch 6/20
625/625 [==============================] - 278s 445ms/step - loss: 0.1067 - accuracy: 0.9610 - val_loss: 0.4404 - val_accuracy: 0.8940
Epoch 7/20
625/625 [==============================] - 275s 441ms/step - loss: 0.0896 - accuracy: 0.9687 - val_loss: 0.4930 - val_accura

2025-06-02 13:40:50.857773: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


782/782 [==============================] - 129s 165ms/step - loss: 0.5904 - accuracy: 0.6736
Test acc: 0.674
